In [1]:
from __future__ import annotations

%load_ext jupyter_black

In [2]:
import os

import numpy as np
import mesoscaler as ms
import matplotlib.pyplot as plt

In [3]:
dx, dy = 200, 200
height, width = 80, 80
levels = [1013.25, 1000, 950, 850, 700, 500, 300, 200]
scale = ms.Mesoscale(200, 175, rate=10, levels=levels)

In [4]:
from mesoscaler.enums import (
    # - ERA5
    GEOPOTENTIAL,
    SPECIFIC_HUMIDITY,
    TEMPERATURE,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
    # - URMA
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
    SURFACE_PRESSURE,
)

_test_data = "../tests/data"

urma_store = os.path.join(_test_data, "urma.zarr")
era5_store = os.path.join(_test_data, "era5.zarr")
# define the variables and levels

era5_dvars = [
    GEOPOTENTIAL,
    TEMPERATURE,
    SPECIFIC_HUMIDITY,
    U_COMPONENT_OF_WIND,
    V_COMPONENT_OF_WIND,
]
urma_dvars = [
    SURFACE_PRESSURE,
    TEMPERATURE_2M,
    SPECIFIC_HUMIDITY_2M,
    U_WIND_COMPONENT_10M,
    V_WIND_COMPONENT_10M,
]
assert len(era5_dvars) == len(urma_dvars), "vard must be the same length in order to be resampled"

dsets = ms.open_datasets([(era5_store, era5_dvars), (urma_store, urma_dvars)], levels=levels)
dsets

DatasetSequence(size=2)[
<xarray.DependentDataset>
Dimensions:              (T: 2, Z: 7, Y: 721, X: 1440)
Coordinates:
    latitude             (Y, X) float32 90.0 90.0 90.0 ... -90.0 -90.0 -90.0
  * vertical             (Z) int64 200 300 500 700 850 950 1000
    longitude            (Y, X) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time                 (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: T, Z, Y, X
Data variables:
    geopotential         (T, Z, Y, X) float32 dask.array<chunksize=(1, 7, 721, 1440), meta=np.ndarray>
    specific_humidity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 7, 721, 1440), meta=np.ndarray>
    temperature          (T, Z, Y, X) float32 dask.array<chunksize=(1, 7, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 7, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 7, 721, 1440), meta=np.ndarray>
Attributes:
    depends:  Dependencies(ERA5)
<xarray.DependentDataset>
Dimensions:               (Y: 1597, X: 2345, T: 2, Z: 1)
Coordinates:
    latitude              (Y, X) float64 19.23 19.23 19.24 ... 54.38 54.38 54.37
    longitude             (Y, X) float64 233.7 233.7 233.8 ... 300.9 300.9 301.0
  * time                  (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
  * vertical              (Z) float64 1.013e+03
Dimensions without coordinates: Y, X, T, Z
Data variables:
    specific_humidity_2m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    surface_pressure      (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    temperature_2m        (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    u_wind_component_10m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    depends:  Dependencies(URMA)
]

In [5]:
area_of_interest = -120, 30, -70, 25
producer = ms.create.producer(
    dsets,
    ms.AreaOfInterestSampler.partial(aoi=area_of_interest, time_batch_size=1),
    height=int(height),
    width=int(width),
    dx=dx,
    dy=dy,
).shuffle(seed=0)

for idx in producer:
    print(producer.get_dataset(idx))
    break

<xarray.Dataset>
Dimensions:              (T: 1, Z: 5, Y: 80, X: 80)
Coordinates:
    vertical             (Z) float64 1.013e+03 850.0 700.0 500.0 300.0
Dimensions without coordinates: T, Z, Y, X
Data variables:
    geopotential         (T, Z, Y, X) float32 nan nan ... 9.503e+04 9.504e+04
    specific_humidity    (T, Z, Y, X) float32 nan nan ... 0.0002509 0.0002468
    temperature          (T, Z, Y, X) float32 nan nan nan ... 241.8 241.2 241.0
    u_component_of_wind  (T, Z, Y, X) float32 nan nan nan ... 15.7 14.81 15.3
    v_component_of_wind  (T, Z, Y, X) float32 nan nan nan nan ... 16.4 15.7 13.7


In [6]:
class DS1Vars(ms.DependentVariables):
    a = ms.auto_field()
    b = ms.auto_field()


class DS2Vars(ms.DependentVariables):
    a = ms.auto_field()
    b = ms.auto_field()


def create_data(
    dvars: type[ms.DependentVariables],
    n: float,
    z: list[int] = [1000, 900, 800, 700, 600, 500, 400, 300, 200, 100],
    t: int = 4,
    x: int = 256,
    y: int = 256,
):
    xx, yy = np.meshgrid(
        np.linspace(0, 360, x),
        np.linspace(0, 180, y),
    )
    start = np.datetime64("2021-01-01")
    time = np.arange(start, start + np.timedelta64(t, "h"), dtype="datetime64[h]")
    return (
        {dvar: (np.zeros((len(time), len(z), x, y)) + i) * n for i, dvar in enumerate(dvars, 1)},
        {ms.vertical: z, ms.time: time, ms.longitude: xx, ms.latitude: yy},
    )


dsets = ms.create.dataset_sequence([create_data(DS1Vars, 0.25)])  # type: ignore
dsets

DatasetSequence(size=1)[
<xarray.DependentDataset>
Dimensions:    (Y: 256, X: 256, T: 4, Z: 10)
Coordinates:
    longitude  (Y, X) float64 0.0 1.412 2.824 4.235 ... 355.8 357.2 358.6 360.0
    latitude   (Y, X) float64 0.0 0.0 0.0 0.0 0.0 ... 180.0 180.0 180.0 180.0
  * time       (T) datetime64[ns] 2021-01-01 ... 2021-01-01T03:00:00
  * vertical   (Z) float64 1e+03 900.0 800.0 700.0 ... 400.0 300.0 200.0 100.0
Dimensions without coordinates: Y, X, T, Z
Data variables:
    a          (T, Z, Y, X) float64 0.25 0.25 0.25 0.25 ... 0.25 0.25 0.25 0.25
    b          (T, Z, Y, X) float64 0.5 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5 0.5 0.5
Attributes:
    depends:  Dependencies(DS1Vars)
]

In [7]:
(ds,) = dsets
time_batch_size = 2
levels = [1000, 900, 800, 700, 600, 500, 400, 300, 200, 100]
dsets = ms.create.dataset_sequence([create_data(DS1Vars, 0.25, z=levels)])  # type: ignore
height = 80
width = 80
producer = ms.create.producer(
    dsets,
    ms.AreaOfInterestSampler.partial(aoi=area_of_interest, time_batch_size=time_batch_size),
    height=80,
    width=80,
    dx=200,
    dy=200,
    levels=levels,
)

x = producer[producer.indices[0]]
assert x.shape == (len(ds.data_vars), time_batch_size, len(levels), height, width)
C, T, Z, Y, X = x.shape
print(x.shape)
assert np.all(x[0] == 0.25)
assert np.all(x[1] == 0.5)

(2, 2, 10, 80, 80)


In [9]:
for batch in ms.create.loader(
    dsets,
    ms.AreaOfInterestSampler.partial(aoi=area_of_interest, time_batch_size=time_batch_size),
    batch_size=1,
):
    print(batch.shape)
    # assert batch.shape == (len(ds.data_vars), time_batch_size, len(levels), height, width)
    # assert np.all(batch[0] == 0.25)
    # assert np.all(batch[1] == 0.5)
    break

torch.Size([1, 2, 2, 3, 80, 80])
